# COSC522 HOMEWORK 5 PYTHON IMPLEMENTATION

# PROBLEM 1 (a)

In [14]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Confusion matrices for L1 and L2
conf_matrix_L1 = np.array([
    [20, 5, 5],
    [3, 24, 3],
    [0, 9, 21]
])

conf_matrix_L2 = np.array([
    [25, 2, 3],
    [3, 22, 5],
    [5, 6, 19]
])

# Class priors (equal distribution as each class has 30 samples)
num_samples_per_class = 30
total_samples = 3 * num_samples_per_class
priors = np.array([num_samples_per_class / total_samples] * 3)  # [P(w1), P(w2), P(w3)]

# Normalize confusion matrices to compute likelihoods P(L1 | w) and P(L2 | w)
likelihoods_L1 = conf_matrix_L1 / np.sum(conf_matrix_L1, axis=1, keepdims=True)
likelihoods_L2 = conf_matrix_L2 / np.sum(conf_matrix_L2, axis=1, keepdims=True)

# Generate the lookup table for Naïve Bayes fusion
lookup_table = []
for l1 in range(3):  # L1 predictions (0-based index)
    for l2 in range(3):  # L2 predictions (0-based index)
        posteriors = []
        for w in range(3):  # True class w
            posterior = likelihoods_L1[w, l1] * likelihoods_L2[w, l2] * priors[w]
            posteriors.append(posterior)
        # Normalize posteriors and choose the class with max posterior probability
        posteriors = np.array(posteriors) / np.sum(posteriors)
        fused_label = np.argmax(posteriors) + 1  # Convert to 1-based indexing
        lookup_table.append([l1 + 1, l2 + 1, fused_label])  # Convert indices to 1-based

# Convert lookup table to DataFrame for better readability
lookup_df = pd.DataFrame(lookup_table, columns=["L1", "L2", "Fused Label"])
print("Naïve Bayes Fusion Lookup Table:")
print(lookup_df)

Naïve Bayes Fusion Lookup Table:
   L1  L2  Fused Label
0   1   1            1
1   1   2            2
2   1   3            1
3   2   1            1
4   2   2            2
5   2   3            3
6   3   1            1
7   3   2            3
8   3   3            3


# PROBLEM 1 (B)

### It is not possible because BKS Table requires joint distribution of the classifiers outputs and true class labels from training data. The confusion matrices do not provide these actual class labels and joint distribution information.

### Difference Between Naive Bayes and BKS is that:

 - Naive Bayes assumes classifier independence and combine probabilities analytically while
 - BKS directly uses observed joint distribution of classifier output without independence assumption

# PROBLEM 1 (C)

### Yes, given a BKS table, I can derive the Naive Bayes look-up table first by assuming or calculating the prior and calculating the marginal probabilities by summing over the observed counts in the BKS table.

# PROBLEM 2 (a)

In [15]:
# Dataset: Square Footage (x) and Rent (y)
x = np.array([750, 800, 850, 900, 950]).reshape(-1, 1)  # Square footage
y = np.array([1160, 1200, 1280, 1450, 2000])  # Rent prices

# L1 Boost using XGBoost
def xgboost_l1_boost(x, y, num_rounds=5):
    """
    Implements L1 Boost using XGBoost.
    """
    dtrain = xgb.DMatrix(x, label=y)
    params = {
        'objective': 'reg:absoluteerror',  # L1 loss
        'learning_rate': 0.8,
        'max_depth': 2,
        'verbosity': 0
    }
    model = xgb.train(params, dtrain, num_boost_round=num_rounds)
    preds = model.predict(dtrain)
    mse = mean_squared_error(y, preds)
    return preds, mse

# L2 Boost using XGBoost
def xgboost_l2_boost(x, y, num_rounds=5):
    """
    Implements L2 Boost using XGBoost.
    """
    dtrain = xgb.DMatrix(x, label=y)
    params = {
        'objective': 'reg:squarederror',  # L2 loss
        'learning_rate': 0.8,
        'max_depth': 2,
        'verbosity': 0
    }
    model = xgb.train(params, dtrain, num_boost_round=num_rounds)
    preds = model.predict(dtrain)
    mse = mean_squared_error(y, preds)
    return preds, mse

# Simple Linear Regression
def simple_linear_regression(x, y):
    """
    Simple Linear Regression implementation using scikit-learn.
    """
    model = LinearRegression()
    model.fit(x, y)
    preds = model.predict(x)
    mse = mean_squared_error(y, preds)
    return preds, mse

# Run L1 Boost
l1_preds, l1_mse = xgboost_l1_boost(x, y, num_rounds=5)
print("Final Predictions (L1 Boost):", l1_preds)
print("Mean Squared Error (L1 Boost):", l1_mse)

# Run L2 Boost
l2_preds, l2_mse = xgboost_l2_boost(x, y, num_rounds=5)
print("Final Predictions (L2 Boost):", l2_preds)
print("Mean Squared Error (L2 Boost):", l2_mse)

# Run Simple Linear Regression
linear_preds, linear_mse = simple_linear_regression(x, y)
print("Final Predictions (Simple Linear Regression):", linear_preds)
print("Mean Squared Error (Simple Linear Regression):", linear_mse)

Final Predictions (L1 Boost): [1160.192  1199.872  1279.872  1571.7888 1878.9888]
Mean Squared Error (L1 Boost): 5895.260768362879
Final Predictions (L2 Boost): [1180.1207 1196.6356 1279.7759 1447.3866 1954.7437]
Mean Squared Error (L2 Boost): 494.2359678119421
Final Predictions (Simple Linear Regression): [1032. 1225. 1418. 1611. 1804.]
Mean Squared Error (Simple Linear Regression): 20077.999999999964


### COMPARISON: MSE for L1 Boost and L2 Boost are both lower than MSE for Simple Linear Regression hence L1 Boost and L2 Boost perform better then Simple Linear Regression 

# PROBLEM 2(B)

It is called gradient boosting because it uses the principle of gradient descent to optimize a loss function during the boosting process. AT each iteration, of the algorithm, the model learns by minimizing either the mean absolute error (for L1 Boost) or mean squared error (for L2Boost).

Boosting is an ensemble technique tha combines multiple weak learniners to ctreate a strong learner.

# PROBLEM 2 (c)

In [16]:
# Dataset: Square Footage (x) and Rent (y)
x = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)  # Square footage
y = np.array([0.07, 0.26, 0.61, 0.87, 0.97])  # Rent prices

# L2 Boost using XGBoost
def xgboost_l2_boost(x, y, num_rounds=5):
    """
    Implements L2 Boost using XGBoost.
    """
    dtrain = xgb.DMatrix(x, label=y)
    params = {
        'objective': 'reg:squarederror',  # L2 loss
        'learning_rate': 0.8,
        'max_depth': 2,
        'verbosity': 0
    }
    model = xgb.train(params, dtrain, num_boost_round=num_rounds)
    preds = model.predict(dtrain)
    mse = mean_squared_error(y, preds)
    return preds, mse

# Simple Linear Regression
def simple_linear_regression(x, y):
    """
    Simple Linear Regression implementation using scikit-learn.
    """
    model = LinearRegression()
    model.fit(x, y)
    preds = model.predict(x)
    mse = mean_squared_error(y, preds)
    return preds, mse

# Run L2 Boost
l2_preds, l2_mse = xgboost_l2_boost(x, y, num_rounds=5)
print("Final Predictions (L2 Boost):", l2_preds)
print("Mean Squared Error (L2 Boost):", l2_mse)

# Run Simple Linear Regression
linear_preds, linear_mse = simple_linear_regression(x, y)
print("Final Predictions (Simple Linear Regression):", linear_preds)
print("Mean Squared Error (Simple Linear Regression):", linear_mse)

Final Predictions (L2 Boost): [0.10891263 0.26561332 0.60602224 0.8729689  0.9393056 ]
Mean Squared Error (L2 Boost): 0.0005024970504785359
Final Predictions (Simple Linear Regression): [0.074 0.315 0.556 0.797 1.038]
Mean Squared Error (Simple Linear Regression): 0.003182000000000003


# PROBLEM 3

In [17]:
def gini_impurity(p):
    """
    Calculate the Gini impurity for a given probability distribution.
    """
    return 1 - sum([pi**2 for pi in p])

def weighted_gini(left_counts, right_counts):
    """
    Calculate the weighted Gini impurity for a split.
    """
    total = sum(left_counts) + sum(right_counts)
    
    # Gini for the left node
    left_total = sum(left_counts)
    left_gini = gini_impurity([count / left_total for count in left_counts]) if left_total > 0 else 0

    # Gini for the right node
    right_total = sum(right_counts)
    right_gini = gini_impurity([count / right_total for count in right_counts]) if right_total > 0 else 0

    # Weighted Gini
    weighted = (left_total / total) * left_gini + (right_total / total) * right_gini
    return weighted

# 3 (a) 
# Option 1: Split information
left_counts_option_1 = [70, 0]  # [Class 1, Class 2]
right_counts_option_1 = [20, 10]  # [Class 1, Class 2]

# 3 (b)
# Option 2: Split information
left_counts_option_2 = [80, 0]  # [Class 1, Class 2]
right_counts_option_2 = [10, 10]  # [Class 1, Class 2]

# Calculate Gini impurity for both options
gini_option_1 = weighted_gini(left_counts_option_1, right_counts_option_1)
gini_option_2 = weighted_gini(left_counts_option_2, right_counts_option_2)

# Print the results
print(f"Gini Impurity for Option 1: {gini_option_1:.4f}")
print(f"Gini Impurity for Option 2: {gini_option_2:.4f}")

# Determine the better query based on Gini impurity
better_option = "Option 1" if gini_option_1 < gini_option_2 else "Option 2"
print(f"Better Query: {better_option}")

Gini Impurity for Option 1: 0.1333
Gini Impurity for Option 2: 0.1000
Better Query: Option 2


# BONUS

# Modifying kNN to Incorporate Prior Probabilities

In standard k-Nearest Neighbors (kNN), the prior probability for a class is fixed at $n_k / n$, where $n_k$ is the number of training samples in class $k$ and $n$ is the total number of training samples. To allow the incorporation of different prior probabilities, the following potential modifications can be made:

---

## 1. Weighted Voting Based on Prior Probabilities
In standard kNN, each neighbor contributes equally to the classification vote. To incorporate prior probabilities, the voting process can be modified so that votes from neighbors belonging to class $c$ are weighted by the desired prior probability $P(c)$.

The score for each class can be calculated as:
$$
\text{Score}(c) = \sum_{i \in \text{neighbors}(c)} \frac{P(c)}{\text{distance}(i, \text{query})^\alpha},
$$
where:
- $P(c)$ is the desired prior for class $c$,
- $\text{distance}(i, \text{query})$ is the distance between neighbor $i$ and the query point,
- $\alpha$ controls the influence of distance on weighting (e.g., $\alpha = 1$).

The query point is assigned the class with the highest score.

---

## 2. Resampling Training Data According to Desired Priors
To simulate different priors, the training dataset can be adjusted by oversampling or undersampling each class. 

For a class $c$ with desired prior $P(c)$:
1. Compute the desired number of samples for class $c$: 
   $$
   n_c^{\text{desired}} = n \cdot P(c),
   $$
   where $n$ is the total number of samples in the training set.
2. Resample the training data for class $c$ to match $n_c^{\text{desired}}$.

This modification alters the class distribution in the training dataset to reflect the desired priors.

---

## 3. Bayesian Adjustment of Posteriors
Bayes' Theorem can be used to adjust the decision-making process to incorporate prior probabilities:
$$
P(c \mid \text{neighbors}) \propto P(c) \cdot P(\text{neighbors} \mid c),
$$
where:
- $P(c)$ is the prior probability for class $c$,
- $P(\text{neighbors} \mid c)$ is the likelihood, proportional to the number of neighbors belonging to class $c$.

### Steps:
1. Count the neighbors belonging to each class.
2. Multiply these counts by the respective class priors $P(c)$.
3. Normalize the adjusted counts to get posterior probabilities:
   $$
   P(c \mid \text{neighbors}) = \frac{P(c) \cdot P(\text{neighbors} \mid c)}{\sum_{k} P(k) \cdot P(\text{neighbors} \mid k)}.
   $$

---

## 4. Adjusting Decision Thresholds
If the output of kNN includes probabilities for each class, the decision threshold can be modified to align with the desired priors. For binary classification:
- The default threshold is $0.5$.
- To incorporate a prior $P(c=1)$, adjust the threshold to:
  $$
  t = P(c=1).
  $$

This approach is simple and does not require modifying the core kNN algorithm.

---

## 5. kNN with Weighted Distance Metrics
The distance metric used in kNN can be adjusted to reflect class priors:
$$
d'(i, \text{query}) = d(i, \text{query}) \cdot \frac{1}{P(c(i))},
$$
where $c(i)$ is the class of neighbor $i$, and $P(c(i))$ is the prior probability for class $c(i)$. 

This adjustment prioritizes neighbors from underrepresented classes and penalizes neighbors from overrepresented classes.

---

## Summary
1. **Weighted Voting** and **Bayesian Adjustment** allow priors to be directly incorporated into kNN's decision-making process.
2. **Resampling Training Data** modifies the dataset to reflect the desired priors.
3. **Threshold Adjustment** is simple and effective for tasks requiring posterior probabilities.
4. **Weighted Distance Metrics** prioritize neighbor selection based on priors.

The choice of method depends on the specific requirements of the task and the ease of implementation.
